In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=0bab00cd40d4912c3bacf9dc9904c34cca2ac5ad566bda46b0ecc768644b65b8
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
# Spar context létrehozása.
# Mindenképp csak egyszer futtassuk le!!!
# Ezen keresztül tudjuk elérni az RDD-kehz tartozó low level API-t.

from pyspark import SparkConf
from pyspark import SparkContext

conf = SparkConf()
sc = SparkContext(conf=conf)

In [3]:
# RDD = resilient distributed dataset
# Az RDD objektumok immutábilis elosztott kollekciója
# RDD létrehozható bármilyen kollekcióból az sc.parallelize() függvénnyel.
# Az sc.parallelize() második argumentumában megadhatjuk hány partíción legyenek elosztva az adatok

tomb = ['a', 'b', 'c', 'd', 'e', 'f']

rdd = sc.parallelize(tomb, 5)
rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:289

In [4]:
# Az alábbi függvény visszaadja az RDD partícióinak a számát.

rdd.getNumPartitions()

5

In [5]:
# Egy RDD újrapraticionálható, azonban ez a klaszter gépei közötti kommunikációt igényel

rdd2 = rdd.repartition(3)
rdd2.getNumPartitions()

3

In [6]:
# Az RDD-ken alkalmazhatunk transzformációkat és akciókat
# A transzformáció RDD-ből új RDD-t hoz létre
# Az akciókkat RDD-re alkalmazzuk és egyszerű értékkel térnek vissza

# Az alábbi kód létrehoz egy RDD-t, amely 1000 számot tartalmaz
# A filter() transzformáció szűr a hárommal osztható számokra
# A count() akicó megszámolja az RDD-ben található értékeket

szamok = sc.parallelize(range(1000)).filter(lambda x: x % 3 == 0)
szamok.count()

334

In [7]:
# Transzformációk: filter, map, flatMap, sort, sample, union, intersection, distinct
# Akciók: count, take, collect, max, min, reduce, foreach
# A nevük elárulja mit csinálnak, de érdemes kipróbálni őket!

# Példa: számok összeadása
# A reduce() akció aggregálja az elemeket egy függvénnyel, amely két argumentumot kap és egy értékkel tér vissza
# Kommutatív és asszociatív függvénynek kell lennie, hogy párhuzamosan végrehajtható legyen

szamok = sc.parallelize(range(1000))\
.filter(lambda x: x % 3 == 0)\
.reduce(lambda a, b: a + b)

print(szamok)

166833


In [9]:
# Lemezen lévő fájlból is készíthetünk RDD-t
# Az alábbi kód kírja a szöveg első két sorát:

lines = sc.textFile('/content/drive/MyDrive/Colab Notebooks/1-2-3/alkotmany.txt').take(2)

print(lines)

['Mi, a magyar nemzet tagjai, az új évezred kezdetén, felelősséggel minden magyarért, kinyilvánítjuk az alábbiakat:', 'Büszkék vagyunk arra, hogy Szent István királyunk ezer évvel ezelőtt szilárd alapokra helyezte a magyar államot, és hazánkat a keresztény Európa részévé tette.']


In [10]:
# Feladat: Grep külön függvénnyel

def filter_row(line):
    grepWord = 'magyar'
    if grepWord in line:
        return True
    else:
        return False


lines = sc.textFile('/content/drive/MyDrive/Colab Notebooks/1-2-3/alkotmany.txt')\
.filter(filter_row)

lines.take(3)

['Mi, a magyar nemzet tagjai, az új évezred kezdetén, felelősséggel minden magyarért, kinyilvánítjuk az alábbiakat:',
 'Büszkék vagyunk arra, hogy Szent István királyunk ezer évvel ezelőtt szilárd alapokra helyezte a magyar államot, és hazánkat a keresztény Európa részévé tette.',
 'Büszkék vagyunk a magyar emberek nagyszerű szellemi alkotásaira.']

In [ ]:
# Feladat: Grep

grepWord = 'magyar'

lines = sc.textFile('alkotmany.txt')\
.filter(lambda line: grepWord in line)

lines.take(4)

['Mi, a magyar nemzet tagjai, az új évezred kezdetén, felelősséggel minden magyarért, kinyilvánítjuk az alábbiakat:',
 'Büszkék vagyunk arra, hogy Szent István királyunk ezer évvel ezelőtt szilárd alapokra helyezte a magyar államot, és hazánkat a keresztény Európa részévé tette.',
 'Büszkék vagyunk a magyar emberek nagyszerű szellemi alkotásaira.',
 'Ígérjük, hogy megőrizzük az elmúlt évszázad viharaiban részekre szakadt nemzetünk szellemi és lelki egységét. Kinyilvánítjuk, hogy a velünk élő nemzetiségek a magyar politikai közösség részei és államalkotó tényezők.']

In [ ]:
lines = sc.textFile('alkotmany.txt')\
.map(lambda l: l.split(" "))

lines.take(2)

[['Mi,',
  'a',
  'magyar',
  'nemzet',
  'tagjai,',
  'az',
  'új',
  'évezred',
  'kezdetén,',
  'felelősséggel',
  'minden',
  'magyarért,',
  'kinyilvánítjuk',
  'az',
  'alábbiakat:'],
 ['Büszkék',
  'vagyunk',
  'arra,',
  'hogy',
  'Szent',
  'István',
  'királyunk',
  'ezer',
  'évvel',
  'ezelőtt',
  'szilárd',
  'alapokra',
  'helyezte',
  'a',
  'magyar',
  'államot,',
  'és',
  'hazánkat',
  'a',
  'keresztény',
  'Európa',
  'részévé',
  'tette.']]

In [ ]:
lines = sc.textFile('alkotmany.txt')\
.flatMap(lambda l: l.split(" "))\
.sortBy(lambda word: word)

lines.take(10)

['1944.',
 '1949.',
 '1956-os',
 '1990.',
 'Alaptörvényünk',
 'Becsüljük',
 'Bízunk',
 'Büszkék',
 'Büszkék',
 'Büszkék']

In [ ]:
# Kulcs-érték párok készítése
# Műveletek kulcs-érték párokon: reduceByKey, groupByKey, countByKey

lines = sc.textFile('alkotmany.txt')\
.flatMap(lambda l: l.split(" "))\
.map(lambda w: (w.lower()[0], w.lower()))

lines.take(10)

[('m', 'mi,'),
 ('a', 'a'),
 ('m', 'magyar'),
 ('n', 'nemzet'),
 ('t', 'tagjai,'),
 ('a', 'az'),
 ('ú', 'új'),
 ('é', 'évezred'),
 ('k', 'kezdetén,'),
 ('f', 'felelősséggel')]

In [12]:
# Feladat: WordCount
# Kulcs alapján aggregáló függvények: reduceByKey, aggregateByKey, groupByKey, sortByKey

lines = sc.textFile('/content/drive/MyDrive/Colab Notebooks/1-2-3/alkotmany.txt')\
.flatMap(lambda l: l.split(" "))\
.map(lambda l: l.replace(':',"").replace(",","").replace(".","").replace("!",""))\
.map(lambda w: (w.lower(), 1))\
.reduceByKey(lambda a, b: a + b)\
.sortBy(lambda x: x[1], False)

lines.take(5)

[('a', 41), ('és', 24), ('az', 19), ('hogy', 17), ('valljuk', 9)]